# Self-querying Elasticsearch

In [1]:
import os
import openai
openai.organization = "org-guDH0aNEdRkxo8bOtlGqBstO"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.schema import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore
import os

#os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings()

cloud_id="30457331e52649c39b16776f8d5bc137:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDJkODJmMzQ4OWQyMjQ4NmNiMWNjMzRhMzJkZTMwMjg3JDkzOWY5OWQxMzZhYTRiNWFiMTcxODg5YzE1OTNkMzdj"

In [3]:
docs = [
    Document(
        page_content="Um grupo de cientistas traz dinossauros de volta e o caos se espalha",
        metadata={"ano": 1993, "avaliacao": 7.7, "genero": "ficção científican"},
    ),
    Document(
        page_content="Leo DiCaprio se perde em um sonho dentro de um sonho dentro de um sonho dentro de um ...",
        metadata={"ano": 2010, "diretor": "Christopher Nolan", "avaliacao": 8.2},
    ),
    Document(
        page_content="Um psicólogo/detetive se perde em uma série de sonhos dentro de sonhos dentro de sonhos e Inception reaproveitou a ideia",
        metadata={"ano": 2006, "diretor": "Satoshi Kon", "avaliacao": 8.6},
    ),
    Document(
        page_content="Um grupo de mulheres de tamanho normal são extremamente saudáveis ​​e alguns homens anseiam por elas",
        metadata={"ano": 2019, "diretor": "Greta Gerwig", "avaliacao": 8.3, "genero": "drama"},
    ),
    Document(
        page_content="Os brinquedos ganham vida e se divertem muito fazendo isso",
        metadata={"ano": 1995, "genero": "animação"},
    ),    
    Document(
        page_content="Robert McCall se sente em casa no sul da Itália, mas descobre que seus amigos estão sob o controle do crime local. À medida que os eventos se tornam mortais, McCall sabe que deve tornar-se o protetor de seus amigos enfrentando a máfia.",
        metadata={"ano": 2023, "diretor": "Antoine Fuqua", "genero": "ação"},    
    ),
    Document(
        page_content="Três homens entram na Zona, três homens saem da Zona",
        metadata={
            "ano": 1979,
            "avaliacao": 9.9,
            "diretor": "Andrei Tarkovsky",
            "genero": "science fiction",
            "avaliacao": 9.9,
        },
    ),
]

i_name="elasticsearch-self-query-demo"


# Ajustamos a ElasticsearchStore para que use o cloud_id
vectorstore = ElasticsearchStore.from_documents(
    docs,
    embedding=OpenAIEmbeddings(),
    index_name=i_name,
    es_cloud_id=cloud_id,
    es_user="elastic",
    es_password='CcP7PB23XsWCSo0NKZbBfd5I'
)

## Creating our self-querying retriever
<hr>

Agora podemos instanciar nosso retriever. Para fazer isso, precisaremos fornecer algumas informações antecipadamente sobre os campos de metadados suportados pelos nossos documentos e uma breve descrição do conteúdo do documento.

In [4]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="genero",
        description="O gênero do filme",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="ano",
        description="O ano em que o filme foi lançado",
        type="integer",
    ),
    AttributeInfo(
        name="diretor",
        description="O nome do diretor do filme",
        type="string",
    ),
    AttributeInfo(
        name="avaliacao", description="Uma classificação de 1 a 10 para o filme", type="float"
    ),
]


In [5]:
document_content_description = "Breve resumo de um filme"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

In [6]:
# This example only specifies a relevant query
retriever.get_relevant_documents("Quais são alguns filmes sobre dinossauros")

/home/dani-boy/.cache/pypoetry/virtualenvs/chat-langchain-kK18R945-py3.10/lib/python3.10/site-packages/langchain/chains/llm.py:280: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='dinossauros' filter=None limit=None


[Document(page_content='Um grupo de cientistas traz dinossauros de volta e o caos se espalha', metadata={'ano': 1993, 'avaliacao': 7.7, 'genero': 'ficção científican'}),
 Document(page_content='Um grupo de cientistas traz dinossauros de volta e o caos se espalha', metadata={'ano': 1993, 'avaliacao': 7.7, 'genero': 'ficção científican'}),
 Document(page_content='Um grupo de cientistas traz dinossauros de volta e o caos se espalha', metadata={'ano': 1993, 'avaliacao': 7.7, 'genero': 'ficção científican'}),
 Document(page_content='Os brinquedos ganham vida e se divertem muito fazendo isso', metadata={'ano': 1995, 'genero': 'animação'})]

In [7]:
# This example specifies a query and a filter
retriever.get_relevant_documents("Greta Gerwig dirigiu algum filme sobre mulheres")

query='mulheres' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='diretor', value='Greta Gerwig') limit=None


[Document(page_content='Um grupo de mulheres de tamanho normal são extremamente saudáveis \u200b\u200be alguns homens anseiam por elas', metadata={'ano': 2019, 'diretor': 'Greta Gerwig', 'avaliacao': 8.3, 'genero': 'drama'}),
 Document(page_content='Um grupo de mulheres de tamanho normal são extremamente saudáveis \u200b\u200be alguns homens anseiam por elas', metadata={'ano': 2019, 'diretor': 'Greta Gerwig', 'avaliacao': 8.3, 'genero': 'drama'}),
 Document(page_content='Um grupo de mulheres de tamanho normal são extremamente saudáveis \u200b\u200be alguns homens anseiam por elas', metadata={'ano': 2019, 'diretor': 'Greta Gerwig', 'avaliacao': 8.3, 'genero': 'drama'})]

In [8]:
# This example specifies a query and a filter
retriever.get_relevant_documents("Que file aborda crimes na Itália")

query='crimes Itália' filter=None limit=None


[Document(page_content='Robert McCall se sente em casa no sul da Itália, mas descobre que seus amigos estão sob o controle do crime local. À medida que os eventos se tornam mortais, McCall sabe que deve tornar-se o protetor de seus amigos enfrentando a máfia.', metadata={'ano': 2023, 'diretor': 'Antoine Fuqua', 'genero': 'ação'}),
 Document(page_content='Robert McCall se sente em casa no sul da Itália, mas descobre que seus amigos estão sob o controle do crime local. À medida que os eventos se tornam mortais, McCall sabe que deve tornar-se o protetor de seus amigos enfrentando a máfia.', metadata={'ano': 2023, 'diretor': 'Antoine Fuqua', 'genero': 'ação'}),
 Document(page_content='Robert McCall se sente em casa no sul da Itália, mas descobre que seus amigos estão sob o controle do crime local. À medida que os eventos se tornam mortais, McCall sabe que deve tornar-se o protetor de seus amigos enfrentando a máfia.', metadata={'ano': 2023, 'diretor': 'Antoine Fuqua', 'genero': 'ação'}),
 

## Filter k
<hr>

Também podemos usar o recuperador de autoconsulta para especificar k: o número de documentos a serem buscados.

Podemos fazer isso passando enable_limit=True para o construtor.

In [9]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [10]:
# This example only specifies a relevant query
retriever.get_relevant_documents("quais são dois filmes sobre dinossauros")

query='dinossauros' filter=None limit=2


[Document(page_content='Um grupo de cientistas traz dinossauros de volta e o caos se espalha', metadata={'ano': 1993, 'avaliacao': 7.7, 'genero': 'ficção científican'}),
 Document(page_content='Um grupo de cientistas traz dinossauros de volta e o caos se espalha', metadata={'ano': 1993, 'avaliacao': 7.7, 'genero': 'ficção científican'})]

In [11]:
# This example specifies a query and a filter
resultado = retriever.get_relevant_documents("Que filme aborda crimes no sul da Itália")

query='crimes no sul da Itália' filter=None limit=None


In [12]:
resultado[0]

Document(page_content='Robert McCall se sente em casa no sul da Itália, mas descobre que seus amigos estão sob o controle do crime local. À medida que os eventos se tornam mortais, McCall sabe que deve tornar-se o protetor de seus amigos enfrentando a máfia.', metadata={'ano': 2023, 'diretor': 'Antoine Fuqua', 'genero': 'ação'})

## Complex queries in Action!
<hr>

Tentamos algumas consultas simples, mas e as mais complexas? Vamos testar algumas consultas mais complexas que utilizam todo o poder do Elasticsearch.

In [13]:
retriever.get_relevant_documents("quais filmes de animação ou comédia foram lançados nos últimos 30 anos sobre brinquedos animados?")

query='brinquedos animados' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genero', value='animação'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genero', value='comédia')]), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='ano', value=1990)]) limit=None


[Document(page_content='Os brinquedos ganham vida e se divertem muito fazendo isso', metadata={'ano': 1995, 'genero': 'animação'})]

In [14]:
retriever.get_relevant_documents("quais filmes de drama ou ação  foram lançados nos últimos 4 anos?")

query=' ' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genero', value='drama'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genero', value='ação')]), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='ano', value=2016)]) limit=None


[Document(page_content='Robert McCall se sente em casa no sul da Itália, mas descobre que seus amigos estão sob o controle do crime local. À medida que os eventos se tornam mortais, McCall sabe que deve tornar-se o protetor de seus amigos enfrentando a máfia.', metadata={'ano': 2023, 'diretor': 'Antoine Fuqua', 'genero': 'ação'}),
 Document(page_content='Um grupo de mulheres de tamanho normal são extremamente saudáveis \u200b\u200be alguns homens anseiam por elas', metadata={'ano': 2019, 'diretor': 'Greta Gerwig', 'avaliacao': 8.3, 'genero': 'drama'})]

In [ ]:
vectorstore.client.indices.delete(index="elasticsearch-self-query-demo")